## 케라스 API를 사용한 사용자 정의 모델 만들기 with 텐서플로 2.3

DLD(Daejeon Learning Day) 2020을 위해 작성된 노트북입니다.

In [1]:
import tensorflow as tf

tf.__version__

'2.3.0'

#### `Sequential()` 클래스와 함수형 API의 관계

`Sequential()`:

시퀀셜 모델에 10개의 유닛을 가진 완전 연결 층을 추가합니다.

In [2]:
seq_model = tf.keras.Sequential()

seq_model.add(tf.keras.layers.Dense(10, input_shape=(100,)))

seq_model.summary()

# seq_model.compile(loss='categorical_crossentropy')
# seq_model.fit(X, y, batch_size=32, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1010      
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________


함수형 API:

함수형 API를 사용할 때는 `Input()`을 사용해 입력의 크기를 정의해야 합니다. 하지만 `InputLayer` 층이 추가되어 있습니다.

In [3]:
inputs = tf.keras.layers.Input(100)
outputs = tf.keras.layers.Dense(10)(inputs)

func_model = tf.keras.Model(inputs, outputs)

func_model.summary()

# func_model.compile(loss='categorical_crossentropy')
# func_model.fit(X, y, batch_size=32, epochs=10)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________


`Input`의 정체는 무엇일까요? 이 함수는 `InputLayer` 클래스의 객체를 만들어 그 결과를 반환합니다.

In [4]:
type(tf.keras.layers.Input)

function

사실 신경망의 입력층은 입력 그 자체입니다. `InputLayer` 객체의 입력 노드 출력을 그대로 `Dense` 층에 주입할 수 있습ㄴ다. 모든 층은 입력과 출력 노드를 정의합니다.

In [5]:
# inputs = tf.keras.layers.Input(100)

input_layer = tf.keras.layers.InputLayer(100)
inputs = input_layer._inbound_nodes[0].outputs

outputs = tf.keras.layers.Dense(10)(inputs)

input_layer_model = tf.keras.Model(inputs, outputs)

input_layer_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________


함수형 API를 사용한 모델은 `layers` 속성에 `InputLayer` 클래스를 포함합니다.

In [9]:
func_model.layers

하지만 시퀀셜 모델은 `layers` 속성에 `InputLayer` 클래스가 보이지 않습니다.

In [11]:
seq_model.layers

모델은 감춰진 `_layers` 속성이 또 있습니다. 여기에서 `InputLayer` 클래스를 확인할 수 있습니다.

In [13]:
seq_model._layers

또는 `_input_layers` 속성에서도 확인할 수 있습니다.

In [14]:
seq_model._input_layers

시퀀셜 모델은 함수형 모델의 특별한 경우입니다. (`Model` --> `Functional` --> `Sequential`)

`Model` 클래스로 만든 `func_model`은 사실 `Functional` 클래스의 객체입니다. `Model` 클래스는 서브클래싱에 사용합니다.

In [21]:
func_model.__class__

tensorflow.python.keras.engine.functional.Functional

#### 사용자 정의 층 만들기

`tf.layers.Layer` 클래스를 상속하고 `build()` 메서드에서 가중치를 만든다음 `call()` 메서드에서 연산을 구현합니다.

In [75]:
class MyDense(tf.keras.layers.Layer):
    
    def __init__(self, units, activation=None, **kwargs):
        # units와 activation 매개변수 외에 나머지 변수를 부모 클래스의 생성자로 전달합니다.
        super(MyDense, self).__init__(**kwargs)
        self.units = units
        # 문자열로 미리 정의된 활성화 함수를 선택합니다.
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        # call() 메서드를 호출할 때 호출됩니다. 가중치 생성을 지연합니다.
        # 가중치와 절편을 생성합니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=[input_shape[-1], self.units],
                                      initializer='glorot_uniform'   # 케라스의 기본 초기화
                                     )
        self.bias = self.add_weight(name='bias',
                                    shape=[self.units],
                                    initializer='zeros')
    
    def call(self, inputs):  # training=None은 training은 배치 정규화나 드롭아웃 같은 경우 사용
        # __call__() 메서드를 호출할 때 호출됩니다.
        # 실제 연산을 수행합니다.
        z = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            return self.activation(z)
        return z        

In [76]:
my_dense = MyDense(10)

In [65]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.reshape(-1, 784) / 255.

In [66]:
X_train.shape

(60000, 784)

In [81]:
inputs = tf.keras.layers.Input(784)
# Layer.__call__() --> MyDense().build() --> Layer.build() --> MyDense().call()
outputs = MyDense(10, activation='softmax')(inputs)

my_dense_model = tf.keras.Model(inputs, outputs)

my_dense_model.summary()

Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
my_dense_25 (MyDense)        (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [82]:
my_dense_model.compile(loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])

my_dense_model.fit(X_train, y_train, batch_size=32, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4430 - accuracy: 0.8804
Epoch 2/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3024 - accuracy: 0.9156
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2892 - accuracy: 0.9207


#### 사용자 정의 모델 만들기

In [98]:
# fit(), predict(), evaluate() 등의 메서드 제공
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.output_layer = MyDense(10, activation='softmax')
    
    def call(self, inputs):
        return self.output_layer(inputs)

In [99]:
my_model = MyModel()

my_model.compile(loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])
my_model.fit(X_train, y_train, batch_size=32, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4400 - accuracy: 0.8824
Epoch 2/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3017 - accuracy: 0.9165
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2894 - accuracy: 0.9213


In [100]:
my_model.summary()

Model: "my_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_30 (MyDense)        multiple                  7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [97]:
my_model.layers

#### 사용자 정의 훈련

#### 사용자 정의 손실 추가하기